In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

url = "https://s3.amazonaws.com/hubway-data/202301-bluebikes-tripdata.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.read_csv(z.open("202301-bluebikes-tripdata.csv"))
df.head()



,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code
0,250,2023-01-01 00:02:54.0800,2023-01-01 00:07:04.4100,36,Copley Square - Dartmouth St at Boylston St,42.349928,-71.077392,21,Prudential Center - 101 Huntington Ave,42.346520,-71.080658,4213,Subscriber,02115
1,489,2023-01-01 00:10:33.3100,2023-01-01 00:18:42.3690,372,Boylston St at Exeter St,42.349589,-71.079468,404,Mass Ave T Station,42.341356,-71.083370,3153,Customer,NaN
2,461,2023-01-01 00:10:49.9130,2023-01-01 00:18:31.1280,372,Boylston St at Exeter St,42.349589,-71.079468,404,Mass Ave T Station,42.341356,-71.083370,3431,Customer,02186
3,1065,2023-01-01 00:13:58.0640,2023-01-01 00:31:43.5090,58,Mugar Way at Beacon St,42.355536,-71.072869,345,Park Plaza at Charles St S.,42.351828,-71.067811,4398,Customer,NaN
4,601,2023-01-01 00:14:02.6130,2023-01-01 00:24:03.8950,50,Boylston St at Berkeley St,42.351142,-71.073292,374,Tremont St at Hamilton Pl,42.356683,-71.061666,7964,Subscriber,02138


## Missing Data

In [2]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print(f'{col} - {round(pct_missing*100)}%')

tripduration - 0%
starttime - 0%
stoptime - 0%
start station id - 0%
start station name - 0%
start station latitude - 0%
start station longitude - 0%
end station id - 0%
end station name - 0%
end station latitude - 0%
end station longitude - 0%
bikeid - 0%
usertype - 0%
postal code - 6%


## Irregular Data

In [5]:
df.boxplot(column="start station name")

KeyError: "None of [Index(['start station name'], dtype='object')] are in the [columns]"

In [6]:
df["start station name"].describe()

count                           140340
unique                             383
top       MIT at Mass Ave / Amherst St
freq                              3450
Name: start station name, dtype: object

In [10]:
df["start station name"].value_counts()

MIT at Mass Ave / Amherst St             3450
Central Square at Mass Ave / Essex St    3050
MIT Vassar St                            2067
MIT Pacific St at Purrington St          2018
Harvard Square at Mass Ave/ Dunster      1945
                                         ... 
Auburndale                                  2
Newton Library                              1
Goodhue St at Grove St                      1
Ross Playground                             1
Community Life Center                       1
Name: start station name, Length: 383, dtype: int64

In [11]:
df["start station name"].unique()

array(['Copley Square - Dartmouth St at Boylston St',
       'Boylston St at Exeter St', 'Mugar Way at Beacon St',
       'Boylston St at Berkeley St', "The Overlook at St. Gabriel's",
       'Boston Landing', 'MIT at Mass Ave / Amherst St',
       'Lafayette Square at Mass Ave / Main St / Columbia St',
       'Charlestown Navy Yard', 'Union Square - Somerville',
       'Dartmouth St at Newbury St', 'Tremont St at E Berkeley St',
       'Seaport Blvd at Sleeper St', 'Albany St at E. Brookline St',
       'Stuart St at Charles St',
       'Prudential Center - 101 Huntington Ave',
       'Sennott Park Broadway at Norfolk Street',
       'Brighton Center - Washington St at Cambridge St',
       'Cross St at Hanover St', 'Tremont St at West St',
       'Union Square - Brighton Ave at Cambridge St',
       'Central Square at Mass Ave / Essex St', 'Clinton St at North St',
       'Dorchester Ave at Gillette Park',
       'Harvard University Radcliffe Quadrangle at Shepard St / Garden St',
  

## Takeaways:
- Some trips are extremely long, the longest being nearly a month! While this is terrible for someone's credit card bill, this isn't particular representative of the larger sample and skews the data. Any rides that are longer than a 12 hours (43,200 seconds) are dropped. 